In [69]:
import pandas as pd
import numpy as np
from deepface import DeepFace
import re
import requests


Mon objectif : 

Informations Personnelles


Nom complet
Date de naissance
Lieu de naissance
Sexe
Nationalité
État civil
Nombre d'enfants et âges
Coordonnées
Adresse résidentielle
Numéros de téléphone
Adresse email
Profils de médias sociaux
Caractéristiques Physiques
Taille
Poids
Couleur des cheveux
Couleur des yeux
Caractéristiques distinctives (cicatrices, tatouages, etc.)
Santé
Groupe sanguin
Historique médical
Allergies
Médicaments actuels
Conditions de santé chroniques
Éducation et Profession
Niveau d'éducation
Historique scolaire/universitaire
Compétences et qualifications
Expérience professionnelle
Titre de poste actuel
Employeur actuel
Historique de salaire
Finances
Revenu
Historique de crédit
Détails de compte bancaire
Investissements
Biens (immobilier, véhicules, etc.)
Préférences et Habitudes
Habitudes alimentaires
Préférences de loisirs
Habitudes de voyage
Préférences de consommation (marques, produits, etc.)
Habitudes de sommeil
Numérique
Historique de navigation
Données de localisation
Historique d'achats en ligne
Préférences de médias sociaux
Historique de recherche
Légal et Administratif
Numéro de sécurité sociale
Passeport/Numéro de carte d'identité
Permis de conduire
Registre des infractions pénales
Historique de vote
Autres
Appartenance à des groupes ou associations
Orientation sexuelle
Croyances religieuses
Opinions politiques
Relations familiales et sociales

In [70]:
# Définition de la classe Person
class Person:
    def __init__(self, nom, prenom, age, sexe, taille, photo,long,lat):
        self.nom = nom
        self.prenom = prenom
        self.age = age
        self.sexe = sexe
        self.taille = taille
        self.photo = photo
        self.emotion = ""
        self.race = ""  
        taille_m = taille / 100  # Convertir la taille en mètres
        self.poids = 22 * (taille_m ** 2)
        self.vivreseul = 0
        self.address = ""
        self.long = long
        self.lat = lat
        self.IRISdata = []
        
    def __str__(self):
        return pd.DataFrame({'Nom': [self.nom], 'Prénom': [self.prenom], 'Age': [self.age], 'Sexe': [self.sexe], 'Taille': [self.taille], 'Photo': [self.photo], 'Emotion': [self.emotion], 'Race': [self.race], 'Poids': [self.poids],'Vivre seul' : [self.vivreseul]}).to_string(index=False)
    
        

Input data

In [ ]:
Nom = "Seidlitz"
Prenom = "Eloi"
Age = 22
Sexe = "Homme"
Taille = 193 #En cm
photo = "data//DSC_0434.JPG"
latitude = 48.86151123046875
longitude = 2.1342475414276123
Person_to_study = Person(Nom, Prenom, Age, Sexe, Taille, photo,longitude,latitude  )
print(Person_to_study)

     Nom Prénom  Age  Sexe  Taille              Photo Emotion Race   Poids  Vivre seul
Seidlitz   Eloi   22 Homme     193 data//DSC_0434.JPG              81.9478           0


Analyse photo

In [72]:
try:
    analysis = DeepFace.analyze(Person_to_study.photo, actions=['age', 'gender', 'race', 'emotion'])
    print("Résultats bruts de l'analyse :", analysis)  # Imprimez l'ensemble de l'objet d'analyse
    analysis = analysis[0]

    age_estime = analysis.get("age", "Non disponible")
    
    Sexe_dict = analysis.get("gender", "Non disponible")
    if Sexe_dict['Man'] > Sexe_dict['Woman']:
        Sexe_estime = "Homme"
    else:
        Sexe_estime = "Femme"
    Person_to_study.race = analysis.get("dominant_race", "Non disponible")
    Person_to_study.emotion = analysis.get("dominant_emotion", "Non disponible")

    

except Exception as e:
    print("Une erreur s'est produite lors de l'analyse de l'image :", e)

Action: emotion: 100%|██████████| 4/4 [00:00<00:00,  4.27it/s]

Résultats bruts de l'analyse : [{'age': 24, 'region': {'x': 3521, 'y': 2069, 'w': 385, 'h': 385}, 'gender': {'Woman': 0.000979610558715649, 'Man': 99.99902248382568}, 'dominant_gender': 'Man', 'race': {'asian': 1.5486403981412877e-05, 'indian': 6.778778356419934e-05, 'black': 1.0187637400349558e-06, 'white': 98.82451295852661, 'middle eastern': 0.4882971290498972, 'latino hispanic': 0.6871028803288937}, 'dominant_race': 'white', 'emotion': {'angry': 3.7639102988887285e-11, 'disgust': 1.3608666875400929e-31, 'fear': 5.798570421992145e-16, 'happy': 99.99682903289795, 'sad': 5.079766698967182e-14, 'surprise': 8.665600355339809e-10, 'neutral': 0.0031747917091706768}, 'dominant_emotion': 'happy'}, {'age': 39, 'region': {'x': 1974, 'y': 610, 'w': 69, 'h': 69}, 'gender': {'Woman': 14.905597269535065, 'Man': 85.09440422058105}, 'dominant_gender': 'Man', 'race': {'asian': 1.7083652317523956, 'indian': 9.229373186826706, 'black': 7.241677492856979, 'white': 23.85774701833725, 'middle eastern': 4

Vérification input data


In [73]:
if np.abs(Age - age_estime) > 5:
    print("L'âge estimé est très différent de l'âge réel.")
if Sexe != Sexe_estime:
    print("Le sexe estimé est différent du sexe réel.")
else:
    print("Le sexe estimé est le même que le sexe réel et l'âge aussi au passage.")

Le sexe estimé est le même que le sexe réel et l'âge aussi au passage.


Prédiction facile

In [74]:
def estimer_poids(taille_cm, imc_cible=22):
    """
    Estime le poids en kilogrammes basé sur la taille en centimètres et un IMC cible.

    :param taille_cm: Taille en centimètres
    :param imc_cible: Indice de masse corporelle cible (par défaut 22)
    :return: Poids estimé en kilogrammes
    """
    taille_m = taille_cm / 100  # Convertir la taille en mètres
    poids_estime = imc_cible * (taille_m ** 2)
    return poids_estime

poids = estimer_poids(Taille)
poids

81.9478

Variable de bases : Travail en France

In [75]:
Taux_de_Chômage = 7,4
habitants = 67162154
salaries = 19770837

tranche_cible_pop = 20787631 #25-49 ans
tranche_cible_emploi = 22862357 #25-54 ans

Import des différents dataset

In [76]:
datacouple = pd.read_excel("data//ip1774.xls", sheet_name="Figure 1")
#datacouple = datacouple.set_index(datacouple.columns[0])
datacouple = datacouple.dropna()
datacouple

,Age,Couples de même sexe Femmes,Couples de sexe différent Femmes,Couples de même sexe Hommes,Couples de sexe différent Hommes,Couples de même sexe ayant le bac Femmes,Couples de sexe différent ayant le bac Femmes,Couples de même sexe ayant le bac Homme,Couples de sexe différent ayant le bac Homme,Couples de même sexe ayant un bac +3 Femme,Couples de sexe différent ayant un bac +3 Femme,Couples de même sexe ayant un bac +3 Homme,Couples de sexe différent ayant un bac +3 Homme,Couples de même sexe Femme ayant un enfant,Couples de sexe différent ayant un enfant
0,De 18 à 19 ans,0.2,6.8,0.2,2.7,73,70,71,61,0,0,0,0,9,26
1,De 20 à 24 ans,0.7,28.4,0.6,16.7,74,74,76,64,35,36,43,28,17,52
2,De 25 à 29 ans,0.9,57.1,1.0,45.0,78,75,76,65,41,36,45,29,31,79
3,De 30 à 34 ans,0.8,70.3,1.1,64.7,78,75,76,65,38,34,42,27,43,90
4,De 35 à 44 ans,0.7,71.5,0.9,71.5,78,71,79,64,39,32,45,27,44,91
5,De 45 à 54 ans,0.5,66.4,0.8,69.9,67,55,67,48,32,22,37,21,26,70
6,De 55 à 64 ans,0.3,64.8,0.5,72.1,60,41,60,38,27,14,31,17,13,23
7,65 ans ou plus,0.1,47.9,0.2,74.8,49,27,49,31,28,9,27,15,6,7
8,Ensemble,0.4,57.2,0.6,63.1,71,53,70,47,33,22,38,21,25,50


In [77]:
personnevivantseul = pd.read_excel("data//demo-couple-pers-seul-log-age.xlsx")
#personnevivantseul = personnevivantseul.set_index(personnevivantseul.columns[0])

personnevivantseul

,Age,Femmes,Hommes,Ensemble
0,15 à 19 ans,7.30,5.90,6.60
1,20 à 24 ans,22.60,21.00,21.80
2,25 à 39 ans,13.30,20.30,16.70
3,40 à 54 ans,12.10,18.80,15.40
4,55 à 64 ans,24.40,21.00,22.80
5,65 à 79 ans,35.40,19.80,28.20
6,80 ans ou plus,61.60,27.20,48.60
7,15 ans ou plus,22.70,19.20,21.00
8,15 ans ou plus (en millions),6.38,4.94,11.31


In [78]:
basecouple = pd.read_excel("data//base-ic-couples-familles-menages-2020.xlsx", sheet_name="IRIS")  
basecouple

,IRIS,Région,Département,Unité urbaine,Commune ou ARM,Libellé commune ou ARM,TRIRIS,Grand quartier,Libellé de l'IRIS,Type d'IRIS,...,Pop mén Pers Réf Autre en 2020 (compl),Familles en 2020 (compl),Fam Couple avec enfant(s) en 2020 (compl),Fam Monoparentales en 2020 (compl),Fam Couple sans enfant en 2020 (compl),Fam 0 enfant moins 25 ans en 2020 (compl),Fam 1 enfant moins 25 ans en 2020 (compl),Fam 2 enfants moins 25 ans en 2020 (compl),Fam 3 enfants moins 25 ans en 2020 (compl),Fam 4 enfants ou plus moins 25 ans en 2020 (compl)
0,IRIS,REG,DEP,UU2020,COM,LIBCOM,TRIRIS,GRD_QUART,LIBIRIS,TYP_IRIS,...,C20_PMEN_CS8,C20_FAM,C20_COUPAENF,C20_FAMMONO,C20_COUPSENF,C20_NE24F0,C20_NE24F1,C20_NE24F2,C20_NE24F3,C20_NE24F4P
1,010010000,84,01,01000,01001,L'Abergement-Clémenciat,ZZZZZZ,0100100,L'Abergement-Clémenciat (commune non irisée),Z,...,9.217454,257.361108,123.624477,23.12718,110.609451,124.519177,65.609777,47.688253,14.591798,4.952104
2,010020000,84,01,01000,01002,L'Abergement-de-Varey,ZZZZZZ,0100200,L'Abergement-de-Varey (commune non irisée),Z,...,0,76.390674,40.14897,4.784281,31.457423,31.457423,14.352843,25.157904,5.422504,0
3,010040101,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Les Pérouses-Triangle d'Activités,H,...,160.24815,358.137077,140.154206,54.008858,163.974014,169.714426,80.332698,66.261859,25.129763,16.698332
4,010040102,84,01,01303,01004,Ambérieu-en-Bugey,ZZZZZZ,0100401,Longeray-Gare,H,...,191.666204,926.880403,355.971473,199.683177,371.225752,429.778632,223.369136,148.568325,94.285942,30.878367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49269,974240101,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Les Thermes,H,...,245.192191,483.71223,198.657555,155.808238,129.246437,202.125511,119.868803,119.663574,18.964431,23.08991
49270,974240102,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,La Ville-Mare Sèche,H,...,346.038115,461.119157,204.249013,155.601768,101.268376,184.6641,133.533097,104.349619,38.572341,0
49271,974240103,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Bras Sec,H,...,86.811077,204.609346,83.704368,66.997937,53.907041,79.836021,61.778865,49.35122,4.955601,8.68764
49272,974240104,04,974,9D201,97424,Cilaos,ZZZZZZ,9742401,Peter Both-Palmiste Rouge,H,...,123.183976,285.991022,162.393063,64.27678,59.321179,85.923189,109.753251,62.67989,14.223323,13.411368


In [79]:
for i in basecouple.columns:
    print(i)

IRIS
Région
Département
Unité urbaine
Commune ou ARM
Libellé commune ou ARM
TRIRIS
Grand quartier
Libellé de l'IRIS
Type d'IRIS
Label de l'IRIS
Ménages en 2020 (compl)
Ménages 1 personne en 2020 (compl)
Ménages Hommes seuls en 2020 (compl)
Ménages Femmes seules en 2020 (compl)
Ménages Autres sans famille en 2020 (compl)
Ménages avec famille(s) en 2020 (compl)
Mén fam princ Couple sans enfant en 2020 (compl)
Mén fam princ Couple avec enfant(s) en 2020 (compl)
Mén fam princ Famille mono en 2020 (compl)
Pop Ménages en 2020 (compl)
Pop mén Personnes seules en 2020 (compl)
Pop mén Hommes seuls en 2020 (compl)
Pop mén Femmes seules en 2020 (compl)
Pop mén Autres sans famille en 2020 (compl)
Pop mén avec famille(s) en 2020 (compl)
Pop mén fam princ Couple sans enfant en 2020 (compl)
Pop mén fam princ Couple avec enfant(s) en 2020 (compl)
Pop mén fam princ Famille mono en 2020 (compl)
Pop 15 ans ou plus en 2020 (princ)
Pop 15-24 ans en 2020 (princ)
Pop 25-54 ans en 2020 (princ)
Pop 55-79 ans e

On part du principe qu'à partir de l'application on puisse récupéré les données de géolocalisation de l'appareil une fois ce qui nous permet de pouvoir avoir des stats sur les couples liée dans sa région. De plus cela nous permettra de pouvoir avoir plus d'information sur les utilisateurs aux alentours

In [82]:
from geopy.geocoders import Nominatim

def get_address_from_coordinates(latitude, longitude):
    # Initialize the Nominatim geocoder
    geolocator = Nominatim(user_agent="geoapiExercises")
    
    # Combine latitude and longitude into a single string
    location = geolocator.reverse(f"{latitude}, {longitude}")
    
    return location.raw["address"]

def get_iris_code_from_coordinates(latitude, longitude):
    # URL de l'API avec les coordonnées passées en paramètres
    api_url = f"https://pyris.datajazz.io/api/coords?lat={latitude}&lon={longitude}"
    try:
        # Effectuer la requête GET vers l'API
        response = requests.get(api_url)
        
        # Vérifier si la requête a réussi (code de statut 200)
        if response.status_code == 200:
            # Extraire les données JSON de la réponse
            data = response.json()
            
            # Extraire le code IRIS
            iris_code = data.get("complete_code")
            
            if iris_code:
                return iris_code
            else:
                return "Code IRIS non trouvé dans la réponse API."
        else:
            return f"Erreur de requête : {response.status_code}"
    except Exception as e:
        return f"Erreur lors de la requête API : {str(e)}"

#address = get_address_from_coordinates(latitude, longitude)
iris_code = get_iris_code_from_coordinates(latitude, longitude)
#print(address)
print(basecouple[basecouple["IRIS"] == iris_code])

            IRIS Région Département Unité urbaine Commune ou ARM  \
40587  780920101     11          78         00851          78092   

      Libellé commune ou ARM  TRIRIS Grand quartier Libellé de l'IRIS  \
40587               Bougival  ZZZZZZ        7809201         Les Quais   

      Type d'IRIS  ... Pop mén Pers Réf Autre en 2020 (compl)  \
40587           H  ...                                61.9188   

      Familles en 2020 (compl) Fam Couple avec enfant(s) en 2020 (compl)  \
40587               588.586538                                263.470229   

      Fam Monoparentales en 2020 (compl)  \
40587                         170.322715   

      Fam Couple sans enfant en 2020 (compl)  \
40587                             154.793594   

      Fam 0 enfant moins 25 ans en 2020 (compl)  \
40587                                206.410767   

      Fam 1 enfant moins 25 ans en 2020 (compl)  \
40587                                180.641356   

      Fam 2 enfants moins 25 ans en 2020

In [83]:
def colonne_age(age):
    column_names = [
        "Pop 15-24 ans vivant seule en 2020 (princ)",
        "Pop 25-54 ans vivant seule en 2020 (princ)",
        "Pop 55-79 ans vivant seule en 2020 (princ)",
        "Pop 80 ans ou plus vivant seule en 2020 (princ)"
    ]
    selected_columns = [column_names[0]]  # Toujours inclure la colonne IRIS
    if age >= 15 and age < 25:
        return column_names[0]
    elif age >= 25 and age < 55:
        return column_names[1]
    elif age >= 55 and age < 80:
         return column_names[2]
    elif age >= 80:
        return column_names[3]



def detect_gender_by_age(df):
    # Dictionnaire pour mapper les tranches d'âge aux colonnes correspondantes
    age_columns_mapping = {
        "Homme": ["Hommes"],
        "Femme": ["Femmes"]
    }

    # Initialisez des listes pour stocker les colonnes correspondantes à l'âge spécifié
    homme_columns = []
    femme_columns = []

    # Parcourez toutes les colonnes du DataFrame
    for column in df.columns:
        for gender, age_keywords in age_columns_mapping.items():
            # Vérifiez si la colonne contient des mots-clés d'âge correspondant à l'âge spécifié
            if any(keyword in column for keyword in age_keywords):
                # Ajoutez la colonne correspondante à la liste appropriée (homme ou femme)
                if gender == "Homme":
                    homme_columns.append(column)
                elif gender == "Femme":
                    femme_columns.append(column)

    # Retournez les listes de colonnes correspondantes à l'âge spécifié pour homme et femme
    return homme_columns, femme_columns

# Exemple d'utilisation :
hommes_correspondants, femmes_correspondantes = detect_gender_by_age(basecouple[basecouple["IRIS"] == iris_code])
print(colonne_age(Person_to_study.age))
populationtotal = basecouple[basecouple["IRIS"] == iris_code]["Pop Ménages en 2020 (compl)"]
print(populationtotal)
popu_veuf = basecouple[basecouple["IRIS"] == iris_code]["Pop 15 ans ou plus veuves ou veufs en 2020 (princ)"]
print(popu_veuf)
popu_divorce = basecouple[basecouple["IRIS"] == iris_code]["Pop 15 ans ou plus divorcée en 2020 (princ)"]
print(popu_divorce)
popu_celib = basecouple[basecouple["IRIS"] == iris_code]["Pop 15 ans ou plus célibataire en 2020 (princ)"]
print(popu_celib)
print(f"Colonnes correspondantes pour Homme ({Person_to_study.age} ans) :")
print(hommes_correspondants)

print(f"Colonnes correspondantes pour Femme ({Person_to_study.age} ans) :")
print(femmes_correspondantes)


Pop 15-24 ans vivant seule en 2020 (princ)
40587    2111.840835
Name: Pop Ménages en 2020 (compl), dtype: object
40587    102.116643
Name: Pop 15 ans ou plus veuves ou veufs en 2020 (princ), dtype: object
40587    121.743908
Name: Pop 15 ans ou plus divorcée en 2020 (princ), dtype: object
40587    463.563751
Name: Pop 15 ans ou plus célibataire en 2020 (princ), dtype: object
Colonnes correspondantes pour Homme (22 ans) :
['Ménages Hommes seuls en 2020 (compl)', 'Pop mén Hommes seuls en 2020 (compl)']
Colonnes correspondantes pour Femme (22 ans) :
['Ménages Femmes seules en 2020 (compl)', 'Pop mén Femmes seules en 2020 (compl)']


In [84]:
import geopandas as gpd
from shapely.geometry import Point

iris_shapes = gpd.read_file('data//contours-iris.geojson')


# Define a tolerance value (in degrees)
tolerance = 0.0001  # Adjust this value as needed

def get_iris_from_coordinates(lon, lat, iris_shapes):
    # Create a point from the coordinates
    point = Point(lon, lat)
    
    # Debugging: Print the point coordinates
    print("Point Coordinates:", point)
    
    # Find the IRIS code that contains the point within the tolerance
    containing_iris = iris_shapes[iris_shapes.geometry.buffer(tolerance).contains(point)]
    
    # Debugging: Print the containing_iris DataFrame
    print("Containing IRIS Data:", containing_iris)
    
    if not containing_iris.empty:
        return containing_iris.iloc[0]['code_iris']
    else:
        return None

def get_person_info_by_iris(iris_code, gender, age, df):
    # Here add the logic to filter by gender and age as shown in previous examples
    pass

# Example usage
lon, lat = 2.2361163, 48.8960224  # Replace with actual coordinates
gender = 'homme'                         # or 'femme'
age = 30                                 # Replace with actual age

# Get the IRIS code from the coordinates
#iris_code = get_iris_from_coordinates(lon, lat, iris_shapes)
iris_code = "920260405"
print(iris_code)
basecouple[basecouple["IRIS"] == iris_code]

920260405


,IRIS,Région,Département,Unité urbaine,Commune ou ARM,Libellé commune ou ARM,TRIRIS,Grand quartier,Libellé de l'IRIS,Type d'IRIS,...,Pop mén Pers Réf Autre en 2020 (compl),Familles en 2020 (compl),Fam Couple avec enfant(s) en 2020 (compl),Fam Monoparentales en 2020 (compl),Fam Couple sans enfant en 2020 (compl),Fam 0 enfant moins 25 ans en 2020 (compl),Fam 1 enfant moins 25 ans en 2020 (compl),Fam 2 enfants moins 25 ans en 2020 (compl),Fam 3 enfants moins 25 ans en 2020 (compl),Fam 4 enfants ou plus moins 25 ans en 2020 (compl)
46522,920260405,11,92,00851,92026,Courbevoie,920841,9202604,Étudiants,H,...,309.636299,609.054039,339.76297,145.963415,123.327654,168.589756,174.450129,174.541379,63.338237,28.134539


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.read_file('iris_geodata.geojson')

def get_iris_from_coords(lat, lon, iris_shapes):
    # Create a point object based on the latitude and longitude
    point = Point(lon, lat)
    # Find the IRIS code for the point
    for _, row in iris_shapes.iterrows():
        if row['geometry'].contains(point):
            return row['IRIS_CODE']
    return None


def get_person_info(iris, gender, age, df):
    # Determine the gender-specific columns we're interested in
    if gender.lower() == 'homme':
        gender_cols = [col for col in df.columns if 'Hommes' in col]
    elif gender.lower() == 'femme':
        gender_cols = [col for col in df.columns if 'Femmes' in col]
    else:
        raise ValueError("Gender must be either 'homme' or 'femme'")
    
    # Determine the age range column
    if age < 25:
        age_col = 'P20_POP1524'
    elif 25 <= age < 55:
        age_col = 'P20_POP2554'
    elif 55 <= age < 80:
        age_col = 'P20_POP5579'
    else:
        age_col = 'P20_POP80P'
    
    # Filter the DataFrame for the given IRIS code
    person_df = df[df['IRIS'] == iris]
    
    # Extract the relevant information
    info = person_df[gender_cols + [age_col]].iloc[0].to_dict()
    return info

# Example usage
iris_code = '123456789'  # Replace with the actual IRIS code
gender = 'homme'         # or 'femme'
age = 30                 # Replace with the actual age

# Get the information
person_info = get_person_info(iris_code, gender, age, df)

# Print or process the `person_info` as needed
print(person_info)

DriverError: iris_geodata.geojson: No such file or directory

Process des différents dataset afin d'avoir plus d'information sur une personne

In [90]:

def age_to_interval(age_str):
    try:
        # Utiliser des expressions régulières pour extraire les nombres
        numbers = re.findall(r'\d+', age_str)
        if age_str == "Ensemble" or "en millions" in age_str:
            return None
        if "plus" in age_str:
            # Gérer les cas comme "65 ans ou plus"
            return pd.Interval(int(numbers[0]), float('inf'), closed='left')
        elif len(numbers) >= 2:
            # Gérer les cas avec deux nombres, comme "15 à 19 ans"
            return pd.Interval(int(numbers[0]), int(numbers[1]), closed='left')
        else:
            raise ValueError("Format d'âge non reconnu")
    except ValueError as e:
        print(f"Erreur avec l'entrée : '{age_str}' - {e}")
        raise


# Vérifier si la colonne "Age" contient déjà des intervalles
if not isinstance(personnevivantseul['Age'].iloc[0], pd.Interval):
    personnevivantseul['Age Range'] = personnevivantseul['Age'].apply(age_to_interval)
else:
    personnevivantseul['Age Range'] = personnevivantseul['Age']

# Fonction pour obtenir les informations de proportion
def get_proportion(age, gender, df):
    # Trouver l'intervalle d'âge
    interval = find_age_interval(age, df)
    if interval == "Âge non trouvé dans les intervalles":
        return interval
    
    # Sélectionner la ligne correspondante à l'intervalle d'âge
    row = df[df['Age Range'] == interval]
    
    # Sélectionner la colonne en fonction du sexe
    if gender.lower() == 'femme':
        proportion = row['Femmes'].values[0]
    elif gender.lower() == 'homme':
        proportion = row['Hommes'].values[0]
    else:
        return "Sexe non reconnu"
    
    # Construire la phrase récapitulative
    return proportion


# Fonction pour trouver l'intervalle d'un âge donné
def find_age_interval(age, df):
    for interval in df['Age Range']:
        if interval.left <= age < interval.right:
            return interval
    return "Âge non trouvé dans les intervalles"


# Exemple d'utilisation

pourcentage_seul = get_proportion(Person_to_study.age, Person_to_study.sexe, personnevivantseul)
print(f"Pour les {Person_to_study.sexe}s âgés de {Person_to_study.age} ans, la proportion vivant seuls est de {pourcentage_seul}%.")

Person_to_study.vivreseul = pourcentage_seul

Pour les Hommes âgés de 22 ans, la proportion vivant seuls est de 21.0%.


Data de couple

In [89]:
if not isinstance(datacouple['Age'].iloc[0], pd.Interval):
    datacouple['Age Range'] = datacouple['Age'].apply(age_to_interval)
else:
    # Si "Age" est déjà une colonne d'intervalles, utilisez-la directement
    datacouple['Age Range'] = datacouple['Age']

def get_data_couple_age(age, gender, df):
    # Adjust the filter to account for both singular and plural forms of "Homme/Hommes"
    gender_str = 'Homme' if gender.lower() == 'homme' else 'Femme'
    
    # Find the age interval
    interval = find_age_interval(age, df)
    if interval == "Âge non trouvé dans les intervalles":
        return interval
    
    # Select the row corresponding to the age interval
    row = df[df['Age Range'] == interval]
    
    # Filter the columns based on gender, allowing for both 'Homme' and 'Hommes'
    gender_columns = [col for col in df.columns if gender_str in col]
    
    # Select only the columns that match the gender
    gender_data = row[gender_columns].iloc[0]  # Use iloc[0] to select the first (and only) row as a Series
    
    return gender_data  # Convert to dictionary for easier readability

# Example usage
age_to_find = Person_to_study.age
gender_to_find = Person_to_study.sexe
data_for_gender = get_data_couple_age(age_to_find, gender_to_find, datacouple)
print(data_for_gender)



Couples de même sexe Hommes                          0.6
Couples de sexe différent Hommes                    16.7
Couples de même sexe  ayant le bac Homme            76.0
Couples de sexe différent  ayant le bac Homme       64.0
Couples de même sexe  ayant un bac +3 Homme         43.0
Couples de sexe différent  ayant un bac +3 Homme    28.0
Name: 1, dtype: float64


Lancement du scrapper linkedin

In [ ]:
#print(asyncio.run(linkedinscrapper.scrapper(Person_to_study.prenom,Person_to_study.nom)))


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [87]:
print(Person_to_study)

     Nom Prénom  Age  Sexe  Taille              Photo Emotion  Race   Poids  Vivre seul
Seidlitz   Eloi   22 Homme     193 data//DSC_0434.JPG   happy white 81.9478        22.6
